# Setup

Import Semantic Kernel SDK from pypi.org

In [1]:
# Note: if using a virtual environment, do not run this cell
%pip install -U semantic-kernel
from semantic_kernel import __version__

__version__

Note: you may need to restart the kernel to use updated packages.


'1.24.0'

Initial configuration for the notebook to run properly.

In [22]:
# Make sure paths are correct for the imports

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

### Configuring the Kernel

Let's get started with the necessary configuration to run Semantic Kernel. For Notebooks, we require a `.env` file with the proper settings for the model you use. Create a new file named `.env` and place it in this directory. Copy the contents of the `.env.example` file from this directory and paste it into the `.env` file that you just created.

**NOTE: Please make sure to include `GLOBAL_LLM_SERVICE` set to either OpenAI, AzureOpenAI, or HuggingFace in your .env file. If this setting is not included, the Service will default to AzureOpenAI.**

#### Option 1: using OpenAI

Add your [OpenAI Key](https://openai.com/product/) key to your `.env` file (org Id only if you have multiple orgs):

```
GLOBAL_LLM_SERVICE="OpenAI"
OPENAI_API_KEY="sk-..."
OPENAI_ORG_ID=""
OPENAI_CHAT_MODEL_ID=""
OPENAI_TEXT_MODEL_ID=""
OPENAI_EMBEDDING_MODEL_ID=""
```
The names should match the names used in the `.env` file, as shown above.

#### Option 2: using Azure OpenAI

Add your [Azure Open AI Service key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio) settings to the `.env` file in the same folder:

```
GLOBAL_LLM_SERVICE="AzureOpenAI"
AZURE_OPENAI_API_KEY="..."
AZURE_OPENAI_ENDPOINT="https://..."
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_TEXT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME="..."
AZURE_OPENAI_API_VERSION="..."
```
The names should match the names used in the `.env` file, as shown above.

For more advanced configuration, please follow the steps outlined in the [setup guide](./CONFIGURING_THE_KERNEL.md).

Let's define our kernel for this example.

In [7]:
from semantic_kernel import Kernel

kernel = Kernel()

We will load our settings and get the LLM service to use for the notebook.

In [24]:
from utils.services import Service 
from utils.service_settings import ServiceSettings

service_settings = ServiceSettings.create()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


We now configure our Chat Completion service on the kernel.

In [25]:
# Remove all services so that this cell can be re-run without restarting the kernel
kernel.remove_all_services()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

# Run a Semantic Function

**Step 3**: Load a Plugin and run a semantic function:


In [7]:
plugin = kernel.add_plugin(parent_directory="../../../prompt_template_samples/", plugin_name="FunPlugin")

In [ ]:
plugin = kernel.add_plugin(parent_directory="../../../prompt_template_samples/", plugin_name="FunPlugin")

from semantic_kernel.functions import KernelArguments

joke_function = plugin["Joke"]

joke = await kernel.invoke(
    joke_function,
    KernelArguments(input="time travel to dinosaur age", style="super silly"),
)
print(joke)

Why did the time traveler bring a suitcase to the dinosaur age?

Because he heard the T-Rex had a "roaring" good time and didn't want to miss out on the "Jurassic" vacation package!


# Agent

In [26]:
import asyncio
from typing import Annotated

from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatHistory, FunctionCallContent, FunctionResultContent
from semantic_kernel.functions import KernelArguments, kernel_function

## Web Agent

In [6]:
! pip install requests_html

     ---------------------------------------- 0.0/50.1 kB ? eta -:--:--
     ---------------------------------------- 50.1/50.1 kB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/84.9 kB ? eta -:--:--
     ---------------------------------------- 84.9/84.9 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/82.9 kB ? eta -:--:--
   ---------------------------------------- 82.9/82.9 kB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/144.2 kB ? eta -:--:--
   ---------------------------------------- 144.2/144.2 kB 8.4 MB/s eta 0:00:00
  Created wheel for websockets: filename=websockets-10.4-cp312-cp312-win_amd64.whl size=100988 sha256=1323550a1c07f3b3e4b125d6c98896dca1a3bd598c362d4fc97cd33d5f1d2933
  Stored in directory: c:\users\vaalt\appdata\local\pip\cache\wheels\80\cf\6d\5d7e4c920cb41925a178b2d2621889c520d648bab487b1d7fd

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
assemblyai 0.35.1 requires websockets>=11.0, but you have websockets 10.4 which is incompatible.
auth0-python 4.7.2 requires cryptography<44.0.0,>=43.0.1, but you have cryptography 42.0.8 which is incompatible.
auth0-python 4.7.2 requires urllib3<3.0.0,>=2.0.7, but you have urllib3 1.26.20 which is incompatible.
microsoft-kiota-http 1.6.6 requires httpx[http2]>=0.28, but you have httpx 0.27.2 which is incompatible.
microsoft-kiota-http 1.6.6 requires urllib3<3.0.0,>=2.2.2, but you have urllib3 1.26.20 which is incompatible.
playwright 1.50.0 requires pyee<13,>=12, but you have pyee 11.1.1 which is incompatible.
realtime 1.0.6 requires websockets<13,>=11, but you have websockets 10.4 which is incompatible.
sagemaker 2.232.2 requires protobuf<5.0,>=3.12, but you have protobuf 5.2

In [42]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id))
    return kernel

In [9]:
! pip install lxml_html_clean

In [6]:
from lxml.html.clean import Cleaner
import lxml
from lxml import etree

In [7]:
from Plugins.web_plugin import NewsFacade

In [41]:
import yaml

def create_chat_completion_agent(kernel: Kernel, definition_file_path: str, plugins: list) -> ChatCompletionAgent:
    with open(definition_file_path, 'r') as file:
        definition = yaml.safe_load(file)

    return ChatCompletionAgent(
        kernel=kernel,
        name=definition['name'],
        plugins = plugins,
        description=definition['description'],
        instructions=definition['instructions']
    )

In [9]:
import asyncio
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import kernel_function

USER_INPUTS = [
    "Hello",
    "What are the latest news about Microsoft?",
]


web_agent = create_chat_completion_agent(
    kernel = _create_kernel_with_chat_completion('web_agent'),
    definition_file_path = "Prompts/web.yaml",
    plugins=[NewsFacade()],
)

# web_agent = ChatCompletionAgent(
#     kernel = _create_kernel_with_chat_completion('web_agent'),
#     #service=AzureChatCompletion(),
#     name="web_agent",
#     instructions=system_message,
#     plugins=[NewsFacade()],
# )

# 2. Create a chat history to hold the conversation
chat_history = ChatHistory()

for user_input in USER_INPUTS:
    # 3. Add the user input to the chat history
    chat_history.add_user_message(user_input)
    print(f"# User: {user_input}")
    # 4. Invoke the agent for a response
    response = await web_agent.get_response(chat_history)
    print(f"# {response.name}: {response.content} ")





# User: Hello
# NewsAgent: Hi! How can I assist you today? Are you looking for any investment news or articles? If so, please provide the ticker symbol you're interested in. 
# User: What are the latest news about Microsoft?
# NewsAgent: Here are the latest news about Microsoft:

1. **Title:** [Magnificent Seven Stocks: Nvidia Sells Off Ahead Of Jensen Huang Speech; Google, Meta, Tesla Tumble](https://finance.yahoo.com/m/4205eaa9-f620-3a0b-a81a-0e82c7c9fd0b/magnificent-seven-stocks%3A.html)
   - **Published:** Today at 01:19 PM

2. **Title:** [Google's Biggest Buy Yet: $32B Wiz Deal Strengthens Cloud Security](https://finance.yahoo.com/news/googles-biggest-buy-yet-32b-154812726.html)
   - **Published:** Today at 11:48 AM

3. **Title:** [Why Alphabet Stock Is Falling Today](https://finance.yahoo.com/m/3d7583ed-43d6-3bcf-b58d-c2182422f70b/why-alphabet-stock-is-falling.html)
   - **Published:** Today at 11:48 AM

4. **Title:** [Google's biggest buy ever](https://finance.yahoo.com/news/goo

## SQL Agent

In [18]:
! pip install faker

   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
    --------------------------------------- 0.0/1.9 MB 325.1 kB/s eta 0:00:06
    --------------------------------------- 0.0/1.9 MB 325.1 kB/s eta 0:00:06
    --------------------------------------- 0.0/1.9 MB 245.8 kB/s eta 0:00:08
   - -------------------------------------- 0.1/1.9 MB 297.7 kB/s eta 0:00:07
   - -------------------------------------- 0.1/1.9 MB 348.6 kB/s eta 0:00:06
   -- ------------------------------------- 0.1/1.9 MB 399.4 kB/s eta 0:00:05
   --- ------------------------------------ 0.2/1.9 MB 458.0 kB/s eta 0:00:04
   ---- ----------------------------------- 0.2/1.9 MB 518.8 kB/s eta 0:00:04
   ----- ---------------------------------- 0.3/1.9 MB 630.9 kB/s eta 0:00:03
   ------- -------------------------------- 0.4/1.9 MB 764.2 kB/s eta 0:00:03
   --------- ------------------------------ 0.5/1.9 MB 893.0 kB/s eta 0:00:02
   -

In [45]:
import sqlite3
import pandas as pd

# Sample data for the climbing product catalog
data = {
    "Product ID": ["P001", "P002", "P003", "P004", "P005"],
    "Product Name": ["Climbing Rope", "Climbing Shoes", "Carabiner", "Harness", "Chalk Bag"],
    "Category": ["Ropes", "Shoes", "Hardware", "Harnesses", "Accessories"],
    "Price": [100, 150, 20, 75, 15],
    "Stock": [50, 30, 100, 40, 200],
    "Description": [
        "Durable and strong climbing rope suitable for all terrains.",
        "High-performance climbing shoes for advanced climbers.",
        "Lightweight and secure carabiner for all climbing needs.",
        "Comfortable and adjustable harness for safety.",
        "Compact and durable chalk bag for better grip."
    ]
}

# Converting the dictionary to a DataFrame
df = pd.DataFrame(data)

# Connecting to a SQLite database (it will create a new one if it doesn't exist)
conn = sqlite3.connect('climbing_product_catalog.db')

# Storing the DataFrame in the SQLite database
df.to_sql('product_catalog', conn, index=False, if_exists='replace')

# Committing and closing the connection
conn.commit()
conn.close()

In [53]:
from Plugins.QueryDb.queryDb import QueryDbPlugin

sql_agent = create_chat_completion_agent(
    kernel = _create_kernel_with_chat_completion('sql_agent'),
    definition_file_path = "Prompts/sql.yaml",
    plugins=[QueryDbPlugin('climbing_product_catalog.db')],
)

In [54]:
# Define the chat history
from semantic_kernel.contents import ChatHistory
chat = ChatHistory()

# Add the user message
chat.add_user_message("how many products are in stock?")


# Generate the agent response
response = await sql_agent.get_response(chat)
response.content

Variable `Symbols.VAR_PREFIX: input` not found in the KernelArguments


'There are a total of 420 products in stock.'

## Concierge Agent

In [57]:
concierge_agent = create_chat_completion_agent(
    kernel = _create_kernel_with_chat_completion('concierge_agent'),
    definition_file_path = "Prompts/concierge.yaml",
    plugins=[]
)

## Cart Agent

In [60]:
from Plugins.cart_plugin import CartPlugin


concierge_agent = create_chat_completion_agent(
    kernel = _create_kernel_with_chat_completion('cart_agent'),
    definition_file_path = "Prompts/cart.yaml",
    plugins=[CartPlugin('http://localhost:3000')]
)

In [ ]:
# Define the chat history
from semantic_kernel.contents import ChatHistory
chat = ChatHistory()

# Add the user message
chat.add_user_message("add climbing shoes to my cart, cost is 100$")


# Generate the agent response
response = await sql_agent.get_response(chat)
response.content

### group chat

In [61]:
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies import TerminationStrategy
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "approved" in history[-1].content.lower()

In [21]:
TASK = "a report on the profitability of investing MSFT stock considering the ticker"

In [41]:
selection_function = KernelFunctionFromPrompt(
    function_name="selection", 
    prompt=f"""
Examine the provided RESPONSE and choose the next participant.
State only the name of the chosen participant without explanation.
Never choose the participant named in the RESPONSE.

Choose only from these participants:
- {web_agent}
- {report_agent}
- {reviewer_agent}

Rules:
- If RESPONSE is user input, it is {web_agent}'s turn.
- If RESPONSE is by {web_agent}, it is {report_agent}'s turn.
- If RESPONSE is by {report_agent}, it is {reviewer_agent}'s turn.
- If RESPONSE is by {reviewer_agent}, it is {report_agent}'s turn.

RESPONSE:
{{{{$lastmessage}}}}
"""
)

In [42]:
termination_keyword = "yes"

termination_function = KernelFunctionFromPrompt(
    function_name="termination", 
    prompt=f"""
Examine the RESPONSE and determine whether the content has been deemed satisfactory.
If the content is satisfactory, respond with a single word without explanation: {termination_keyword}.
If specific suggestions are being provided, it is not satisfactory.
If no correction is suggested, it is satisfactory.

RESPONSE:
{{{{$lastmessage}}}}
"""
)

In [43]:
import asyncio
import os

from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatHistoryTruncationReducer
from semantic_kernel.functions import KernelFunctionFromPrompt

history_reducer = ChatHistoryTruncationReducer(target_count=1)

In [44]:
agents=[web_agent, report_agent, reviewer_agent]

chat = AgentGroupChat(
    agents=agents,
    selection_strategy=KernelFunctionSelectionStrategy(
        initial_agent=web_agent,
        function=selection_function,
        kernel=_create_kernel_with_chat_completion('selection_function'),
        result_parser=lambda result: str(result.value[0]).strip() if result.value[0] is not None else WRITER_NAME,
        history_variable_name="lastmessage",
        history_reducer=history_reducer,
    ),
    termination_strategy=KernelFunctionTerminationStrategy(
        agents=[reviewer_agent],
        function=termination_function,
        kernel=_create_kernel_with_chat_completion('termination_function'),
        result_parser=lambda result: termination_keyword in str(result.value[0]).lower(),
        history_variable_name="lastmessage",
        maximum_iterations=10,
        history_reducer=history_reducer,
    ),
)

In [45]:
import asyncio

await chat.add_chat_message(message=TASK)
print(f"# User: {TASK}")

async def main():
    async for content in chat.invoke():
        print(f"# {content.name}: {content.content}")

await main() 

# User: a report on the profitability of investing MSFT stock considering the ticker
# web_agent: ### Report on the Profitability of Investing in MSFT Stock

#### Recent News and Developments:

1. **Article:** [Trump Trade: Binance founder denies report on Trump talks](https://finance.yahoo.com/news/trump-trade-binance-founder-denies-140515998.html)
   - **Date:** Today, 10:05 AM
   - **Summary:** This article discusses recent market speculation involving Binance and Trump, though it doesn't directly relate to Microsoft (MSFT), it highlights market dynamics that can indirectly affect stocks like MSFT.

2. **Article:** [This Nvidia-Backed Company Is Seeking a $35 Billion Valuation in One of the Most Eagerly Anticipated IPOs of 2025: 3 Things You Should Know](https://finance.yahoo.com/m/36949ba6-510d-3e1f-96ff-4519e7a1f5b4/this-nvidia-backed-company-is.html)
   - **Date:** Today, 07:20 AM
   - **Summary:** The anticipated IPO of an Nvidia-backed company sets a competitive and innovative 

In [36]:
agents=[web_agent, report_agent, reviewer_agent]

agent_group_chat = AgentGroupChat(
        agents=agents,
        selection_strategy=create_selection_strategy(_create_kernel_with_chat_completion('default'), agents, report_agent),
        termination_strategy =ApprovalTerminationStrategy(
                agents=[reviewer_agent],
                maximum_iterations=10,
    ))

await group_chat.add_chat_message(message=TASK)
print(f"# User: {TASK}")

# User: a report on the profitability of investing MSFT stock considering the ticker


In [37]:
import asyncio

async def main():
    async for content in agent_group_chat.invoke():
        print(f"# {content.name}: {content.content}")

await main() 

Failed to select agent: 'FunctionResult' object is not subscriptable


AgentChatException: Failed to select agent

## Group Chat

In [22]:
# 3. Place the agents in a group chat with a custom termination strategy
group_chat = AgentGroupChat(
    agents=[
        web_agent,
        report_agent,
        reviewer_agent,
    ],
    termination_strategy=ApprovalTerminationStrategy(
        agents=[reviewer_agent],
        maximum_iterations=10,
    ),
)

# 4. Add the task as a message to the group chat
await group_chat.add_chat_message(message=TASK)
print(f"# User: {TASK}")


# User: a report on the profitability of investing MSFT stock considering the ticker


In [23]:
import asyncio

async def main():
    async for content in group_chat.invoke():
        print(f"# {content.name}: {content.content}")

await main() 

# web_agent: ## Investment Report: MSFT Stock

### Recent News and Articles

1. **Title:** [This Nvidia-Backed Company Is Seeking a $35 Billion Valuation in One of the Most Eagerly Anticipated IPOs of 2025: 3 Things You Should Know](https://finance.yahoo.com/m/36949ba6-510d-3e1f-96ff-4519e7a1f5b4/this-nvidia-backed-company-is.html)
   - **Source:** Yahoo Finance
   - **Date:** Today, 07:20 AM
   - **Summary:** This article discusses a Nvidia-backed company's upcoming IPO, which has implications for Microsoft due to potential collaborations and technological advancements.

2. **Title:** [3 Vanguard ETFs to Buy With $1,000 and Hold Forever](https://finance.yahoo.com/m/6ce2983e-a262-37bb-a9b9-4dd7d6d75f1f/3-vanguard-etfs-to-buy-with.html)
   - **Source:** Yahoo Finance
   - **Date:** Today, 04:44 AM
   - **Summary:** This article highlights investment strategies involving Vanguard ETFs, which might include Microsoft within their technology sector holdings, contributing to stable long-term

In [ ]:
kernel.remove_all_services()

service_id = "agent"
kernel = Kernel()
kernel.add_plugin(bing, plugin_name="bing")

In [17]:
kernel.remove_all_services()

service_id = "agent"
kernel = Kernel()
kernel.add_plugin(bing, plugin_name="bing")
kernel.add_service(AzureChatCompletion(service_id=service_id))

# 2. Configure the function choice behavior to auto invoke kernel functions
# so that the agent can automatically execute the menu plugin functions when needed
settings = kernel.get_prompt_execution_settings_from_service_id(service_id=service_id)
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

agent = ChatCompletionAgent(
    kernel=kernel,
    name="MarketingAnalyst",
    instructions="You are a helpful AI Assistant. You navigate the web to find the most relevant information for the user. Always check the web before answering.",
    arguments=KernelArguments(settings=settings),
)

# 4. Create a chat history to hold the conversation
chat_history = ChatHistory()


In [20]:
USER_INPUTS = [
    "Hello",
    "Can you provide me with a detailed analysis of the current stock price of MSFT?",
    "Search for the exact stock price",
]

for user_input in USER_INPUTS:
    # 5. Add the user input to the chat history
    chat_history.add_user_message(user_input)
    print(f"# User: {user_input}")
    # 6. Invoke the agent for a response
    async for content in agent.invoke(chat_history):
        print(f"# {content.name}: ", end="")
        if (
            not any(isinstance(item, (FunctionCallContent, FunctionResultContent)) for item in content.items)
            and content.content.strip()
        ):
            # We only want to print the content if it's not a function call or result
            print(f"{content.content}", end="", flush=True)
    print("")

# User: Hello
# MarketingAnalyst: Hi! How can I assist you today?
# User: Can you provide me with a detailed analysis of the current stock price of MSFT?
# MarketingAnalyst: I'm unable to get the details directly from the search. I suggest checking a financial news website or stock market platform like Yahoo Finance, Bloomberg, or Google Finance for an in-depth analysis of Microsoft's stock price.
# User: Search for the exact stock price
# MarketingAnalyst: I found the latest stock price for Microsoft (MSFT). You can view the most current information on financial news websites or stock market platforms such as:

- Yahoo Finance
- Google Finance
- Bloomberg
- MarketWatch

Please visit these links to get the exact stock price and detailed analysis.

Is there anything else you need help with?
